# CS 287 - HW 4 - Cont.

In [ ]:
import random
import torch
import torch.nn as nn 
import torch.nn.functional as F
import torchtext
from torchtext.vocab import Vectors, GloVe
from namedtensor import ntorch, NamedTensor
from namedtensor.text import NamedField

In [ ]:
# load data
TEXT = NamedField(names=('seqlen',)) # Our input $x$
LABEL = NamedField(sequential=False, names=()) # Our labels $y$
train, val, test = torchtext.datasets.SNLI.splits(TEXT, LABEL)
print('len(train)', len(train))
TEXT.build_vocab(train)
LABEL.build_vocab(train)
print('len(TEXT.vocab)', len(TEXT.vocab))
print('len(LABEL.vocab)', len(LABEL.vocab))
train_iter, val_iter, test_iter = torchtext.data.BucketIterator.splits(
    (train, val, test), batch_size=16, device=torch.device("cuda"), repeat=False)

In [ ]:
# build the vocabulary with word embeddings
# out-of-vocabulary words are hashed to one of 100 random embeddings each initialized to mean 0, stdev 1 (Sec 5.1)
unk_vectors = [torch.randn(300) for _ in range(100)]
TEXT.vocab.load_vectors(vectors='glove.6B.300d', unk_init=lambda x:random.choice(unk_vectors))
vectors = TEXT.vocab.vectors
vectors = vectors / vectors.norm(dim=1, keepdim=True) # normalized to have l_2 norm of 1
vectors = NamedTensor(vectors, ('word', 'embedding'))
TEXT.vocab.vectors = vectors
print("word embeddings shape:", TEXT.vocab.vectors.shape)

## Decomposable Intra-Sentence Attention Model

In [ ]:
[LABEL.vocab.itos[i] for i in [0,1,2,3]]

In [ ]:
[TEXT.vocab.itos[i] for i in [0,1,2,3]]

In [ ]:
# checking to make sure no <unk> labels
total = 0
for batch in iter(train_iter):
    total += torch.sum(batch.label.values == 0)
total

In [ ]:
# no prepend with NULL!
for batch in iter(train_iter):
    print([TEXT.vocab.itos[i] for i in batch.premise.values[:,0]])
    print([TEXT.vocab.itos[i] for i in batch.hypothesis.values[:,0]])
    break

In [ ]:
TEXT.vocab.stoi['null']

In [ ]:
# dimensions
input_size = TEXT.vocab.vectors.shape['word']
embed_size = TEXT.vocab.vectors.shape['embedding']
hidden_size1 = 200
hidden_size2 = hidden_size1 * 2
output_size = len(LABEL.vocab)
print('DIMENSIONS -- input: %d, embed: %d, hidden1: %d, hidden2: %d, output: %d'%(input_size, embed_size, hidden_size1, hidden_size2, output_size))

In [ ]:
# pre-trained embeddings
weights = TEXT.vocab.vectors.values.cuda()
weights.shape

In [ ]:
class EmbedProject(torch.nn.Module):
    def __init__(self, weights, embed_size, project_size):
        super(EmbedProject, self).__init__()
        self.embed = nn.Embedding.from_pretrained(weights, freeze=True) # weights: input_size x embed_size
        self.linear = nn.Linear(embed_size, project_size)
        torch.nn.init.normal_(self.linear.weight, mean=0, std=0.01)
    def forward(self, inputs):
        embedding = self.embed(inputs)
        output = self.linear(embedding)
        return output

In [ ]:
EP1 = EmbedProject(weights, embed_size, hidden_size1).cuda()
EP1

In [ ]:
batch = next(iter(train_iter))
raw_sent1 = batch.premise.values.transpose(0,1)
raw_sent2 = batch.hypothesis.values.transpose(0,1)
null_tkn = torch.tensor(TEXT.vocab.stoi['null'], device='cuda')
null_tkns = null_tkn.repeat(raw_sent1.shape[0],1)
sent1 = torch.cat((null_tkns, raw_sent1), 1)
sent2 = torch.cat((null_tkns, raw_sent2), 1)
raw_sent1.shape, raw_sent2.shape, sent1.shape, sent2.shape

In [ ]:
proj1 = EP1(sent1)
proj2 = EP1(sent2)
proj1.shape, proj2.shape

In [ ]:
class FeedForwardFIntra(torch.nn.Module):
    def __init__(self, input_size, hidden_size, output_size, dropout=0.2):
        super(FeedForwardFIntra, self).__init__()
        self.d = nn.Dropout(dropout)
        self.m = nn.ReLU()
        self.linear1 = nn.Linear(input_size, hidden_size)
        self.linear2 = nn.Linear(hidden_size, output_size)
        for param in self.parameters():
            torch.nn.init.normal_(param, mean=0, std=0.01)
    def forward(self, inputs):
        hidden = self.m(self.linear1(self.d(inputs)))
        output = self.m(self.linear2(self.d(hidden)))
        return output

In [ ]:
FI1 = FeedForwardFIntra(hidden_size1, hidden_size1, hidden_size1).cuda()
FI1

In [ ]:
fi1 = FI1(proj1)
fi2 = FI1(proj2)
fi1.shape, fi2.shape

In [ ]:
# intra-sentence attention!
score1 = torch.bmm(fi1, fi1.transpose(1,2))
score2 = torch.bmm(fi2, fi2.transpose(1,2))
score1.shape, score2.shape

In [ ]:
prob1 = F.softmax(score1, dim=2)
prob2 = F.softmax(score2, dim=2)
prob1.shape, prob2.shape

In [ ]:
# intra-sentence attention!
proj1_soft = torch.bmm(prob1, proj1)
proj2_soft = torch.bmm(prob2, proj2)
proj1_soft.shape, proj2_soft.shape

In [ ]:
# intra-sentence attention!
proj1_intra = torch.cat((proj1, proj1_soft), dim=2)
proj2_intra = torch.cat((proj2, proj2_soft), dim=2)
proj1_intra.shape, proj2_intra.shape

In [ ]:
class FeedForwardF(torch.nn.Module):
    def __init__(self, input_size, hidden_size, output_size, dropout=0.2):
        super(FeedForwardF, self).__init__()
        self.d = nn.Dropout(dropout)
        self.m = nn.ReLU()
        self.linear1 = nn.Linear(input_size, hidden_size)
        self.linear2 = nn.Linear(hidden_size, output_size)
        for param in self.parameters():
            torch.nn.init.normal_(param, mean=0, std=0.01)
    def forward(self, inputs):
        hidden = self.m(self.linear1(self.d(inputs)))
        output = self.m(self.linear2(self.d(hidden)))
        return output

In [ ]:
F1 = FeedForwardF(hidden_size2, hidden_size1, hidden_size1).cuda()
F1

In [ ]:
class FeedForwardG(torch.nn.Module):
    def __init__(self, input_size, hidden_size, output_size, dropout=0.2):
        super(FeedForwardG, self).__init__()
        self.d = nn.Dropout(dropout)
        self.m = nn.ReLU()
        self.linear1 = nn.Linear(input_size, hidden_size)
        self.linear2 = nn.Linear(hidden_size, output_size)
        for param in self.parameters():
            torch.nn.init.normal_(param, mean=0, std=0.01)
    def forward(self, inputs):
        hidden = self.m(self.linear1(self.d(inputs)))
        output = self.m(self.linear2(self.d(hidden)))
        return output

In [ ]:
class FeedForwardH(torch.nn.Module):
    def __init__(self, input_size, hidden_size, output_size, dropout=0.2):
        super(FeedForwardH, self).__init__()
        self.d = nn.Dropout(dropout)
        self.m = nn.ReLU()
        self.linear1 = nn.Linear(input_size, hidden_size)
        self.linear2 = nn.Linear(hidden_size, hidden_size)
        self.linear3 = nn.Linear(hidden_size, output_size)
        for param in self.parameters():
            torch.nn.init.normal_(param, mean=0, std=0.01)
    def forward(self, inputs):
        hidden1 = self.m(self.linear1(self.d(inputs)))
        hidden2 = self.m(self.linear2(self.d(hidden1)))
        output = self.linear3(hidden2)
        return output

In [ ]:
class EmbedDist(torch.nn.Module):
    def __init__(self, num_embeddings, embedding_dim): # num = 11, dim = 1
        super(EmbedDist, self).__init__()
        self.embed = nn.Embedding(num_embeddings, embedding_dim)
        torch.nn.init.normal_(self.embed.weight, mean=0, std=0.01)
    def forward(self, inputs):
        output = self.embed(inputs)
        return output

In [ ]:
dist = 10
seqlen = score1.shape[2]
steps = torch.arange(0, seqlen)
mat_steps = steps.repeat(seqlen, 1)
flip_steps = torch.flip(steps, [0]).view(-1, 1)
idx = torch.min(torch.abs(mat_steps - flip_steps), torch.tensor(dist))

ED1 = EmbedDist(dist+1, 1).cuda()
ED1(idx).squeeze().shape

## Train

In [ ]:
def get_dist_bias(seqlen, dist, ED1):
    steps = torch.arange(0, seqlen)
    mat_steps = steps.repeat(seqlen, 1)
    flip_steps = torch.flip(steps, [0]).view(-1, 1)
    idx = torch.min(torch.abs(mat_steps - flip_steps), torch.tensor(dist))
    return ED1(idx).squeeze().cuda()

In [ ]:
def get_output(sent1, sent2, EP1, F1, G1, H1, intra, dist, FI1, ED1):
    proj1 = EP1(sent1)
    proj2 = EP1(sent2)
    
    mask1 = (sent1 == pad_tkn)
    mask2 = (sent2 == pad_tkn)
        
    if intra:
        fi1 = FI1(proj1)
        fi2 = FI1(proj2)
        score1 = torch.bmm(fi1, fi1.transpose(1,2))
        score2 = torch.bmm(fi2, fi2.transpose(1,2))
        score1 += get_dist_bias(score1.shape[2], dist, ED1)
        score2 += get_dist_bias(score2.shape[2], dist, ED1)
        mask1c = mask1.unsqueeze(1).expand(-1, sent1.shape[1], -1).float()
        mask2c = mask2.unsqueeze(1).expand(-1, sent2.shape[1], -1).float()
        score1 = score1 * (1 - mask1c) + (mask1c * -1e8)
        score2 = score2 * (1 - mask2c) + (mask2c * -1e8)
        prob1 = F.softmax(score1, dim=2)
        prob2 = F.softmax(score2, dim=2)
        proj1_soft = torch.bmm(prob1, proj1)
        proj2_soft = torch.bmm(prob2, proj2)
        proj1 = torch.cat((proj1, proj1_soft), dim=2)
        proj2 = torch.cat((proj2, proj2_soft), dim=2) 
        
    f1 = F1(proj1)
    f2 = F1(proj2)
    
    score1 = torch.bmm(f1, f2.transpose(1,2))
    score2 = score1.transpose(1,2)
    mask1a = mask1.unsqueeze(1).expand(-1, sent2.shape[1], -1).float()
    mask2a = mask2.unsqueeze(1).expand(-1, sent1.shape[1], -1).float()
    score1 = score1 * (1 - mask2a) + (mask2a * -1e8)
    score2 = score2 * (1 - mask1a) + (mask1a * -1e8)
    
    prob1 = F.softmax(score1, dim=2)
    prob2 = F.softmax(score2, dim=2)
    proj1_soft = torch.bmm(prob2, proj1)
    proj2_soft = torch.bmm(prob1, proj2)
    proj1_combined = torch.cat((proj1, proj2_soft), dim=2)
    proj2_combined = torch.cat((proj2, proj1_soft), dim=2)
    
    g1 = G1(proj1_combined)
    g2 = G1(proj2_combined)
    mask1b = mask1.unsqueeze(2).expand(-1, -1, hidden_size1).float()
    mask2b = mask2.unsqueeze(2).expand(-1, -1, hidden_size1).float()
    g1 = g1 * (1 - mask1b)
    g2 = g2 * (1 - mask2b)
    
    g1_sum = g1.sum(dim=1)
    g2_sum = g2.sum(dim=1)
    g_all = torch.cat((g1_sum, g2_sum), dim=1)
    h_all = H1(g_all)
    return h_all

In [ ]:
def prepend_null(sent):
    null_tkns = null_tkn.repeat(sent.shape[0], 1)
    return torch.cat((null_tkns, sent), 1)

In [ ]:
def training_loop(e, train_iter, EP1, F1, G1, H1, criterion, optimizer, intra=False, dist=None, FI1=None, ED1=None):
    EP1.train()
    F1.train()
    G1.train()
    H1.train()
    if intra:
        FI1.train()
        ED1.train()
    
    for ix,batch in enumerate(train_iter):
        optimizer.zero_grad()
        sent1 = prepend_null(batch.premise.values.transpose(0,1))
        sent2 = prepend_null(batch.hypothesis.values.transpose(0,1))
        target = batch.label.values
        output = get_output(sent1, sent2, EP1, F1, G1, H1, intra, dist, FI1, ED1)
        
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        
        if ix % 1000 == 0:
            acc = torch.sum(torch.argmax(output, dim=1) == target).item() / target.shape[0]
            print('Epoch: {0}, Batch: {1}, Train NLL: {2:0.4f}, Train Acc:{3:0.4f}'.format(e, ix, loss.cpu().detach(), acc))

In [ ]:
def validation_loop(e, val_iter, EP1, F1, G1, H1, criterion, intra=False, dist=None, FI1=None, ED1=None):
    EP1.eval()
    F1.eval()
    G1.eval()
    H1.eval()
    if intra:
        FI1.eval()
        ED1.eval()
    
    total_loss = 0
    total_sent = 0
    total_correct = 0
    
    for ix,batch in enumerate(val_iter):
        sent1 = prepend_null(batch.premise.values.transpose(0,1))
        sent2 = prepend_null(batch.hypothesis.values.transpose(0,1))
        target = batch.label.values
        output = get_output(sent1, sent2, EP1, F1, G1, H1, intra, dist, FI1, ED1)
        
        loss = criterion(output, target).item()
        sent = sent1.shape[0]
        correct = torch.sum(torch.argmax(output, dim=1) == target).item()
        
        total_loss += loss*sent
        total_sent += sent
        total_correct += correct
    
    print('Epoch: {0}, Val NLL: {1:0.4f}, Val Acc: {2:0.4f}'.format(e, total_loss/total_sent, total_correct/total_sent))
    return total_loss

In [ ]:
best_loss = 1e8
intra = True

if intra:
    F1 = FeedForwardF(hidden_size2, hidden_size1, hidden_size1).cuda()
    FI1 = FeedForwardFIntra(hidden_size1, hidden_size1, hidden_size1).cuda()
    dist = 10
    num_embeddings = dist + 1
    embedding_dim = 1
    ED1 = EmbedDist(num_embeddings, embedding_dim).cuda()
else:
    F1 = FeedForwardF(hidden_size1, hidden_size1, hidden_size1).cuda()
    FI1 = None
    dist = None
    ED1 = None

EP1 = EmbedProject(weights, embed_size, hidden_size1).cuda()
G1 = FeedForwardG(hidden_size2, hidden_size1, hidden_size1).cuda()
H1 = FeedForwardH(hidden_size2, hidden_size1, output_size).cuda()

parameters = [param for param in EP1.parameters()] # embed, lnr, bias
parameters.extend([param for param in F1.parameters()]) # lnr1, bias1, lnr2, bias2
parameters.extend([param for param in G1.parameters()]) # lnr1, bias1, lnr2, bias2
parameters.extend([param for param in H1.parameters()]) # lnr1, bias1, lnr2, bias2, lnr3, bias3
if intra:
    parameters.extend([param for param in FI1.parameters()]) # lnr1, bias1, lnr2, bias2
    parameters.extend([param for param in ED1.parameters()]) # embed

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adagrad(parameters, lr=0.025, initial_accumulator_value=0.1)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="min", patience=4)

for e in range(100):
    training_loop(e, train_iter, EP1, F1, G1, H1, criterion, optimizer, intra=intra, dist=dist, FI1=FI1, ED1=ED1)
    loss = validation_loop(e, val_iter, EP1, F1, G1, H1, criterion, intra=intra, dist=dist, FI1=FI1, ED1=ED1)
    scheduler.step(loss)
    if loss < best_loss:
        torch.save(EP1.state_dict(),'best_EP1_intra_v02.pt')
        torch.save(F1.state_dict(),'best_F1_intra_v02.pt')
        torch.save(G1.state_dict(),'best_G1_intra_v02.pt')
        torch.save(H1.state_dict(),'best_H1_intra_v02.pt')
        best_loss = loss
        print('WROTE MODEL')